# Intro

## Load

### Standards

In [28]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [1]:
import os, sys, pickle
import numpy as np
from tqdm.notebook import trange, tqdm
import re

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 2

plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1


In [3]:
from multiprocess import Pool

### Homemade modules

In [4]:
from tesste import tesste
from ppp import ppp
from readers_and_converters import file2text, text2dict, text2vec

## Stoopid infos

In [5]:
L_SDGS=17

# Data

In [6]:
with open ('./data/energy_utilities_texts.pickle', 'rb') as f:
    text_dict=pickle.load(f)

# Semantic analysis

## Cosine similarity calculations

In [7]:
cos_sim_dict={}
for year in range(2015, 2024):
    s_year=str(year)
    cos_sim_dict[year]={}
    energy_co_s=list(text_dict['Energy'][s_year].keys())
    # inside energy
    energy_cos_sims=np.zeros((len(energy_co_s), len(energy_co_s)))
    energy_pvals=np.zeros((len(energy_co_s), len(energy_co_s)))
    for i_e, energy_co_0 in enumerate(tqdm(energy_co_s, leave=False, desc='Energy '+s_year)):
        text_0=text_dict['Energy'][s_year][energy_co_0]
        for j_e in range(i_e+1, len(energy_co_s)):
            energy_co_1=energy_co_s[j_e]
            text_1=text_dict['Energy'][s_year][energy_co_1]
            cacca=tesste(text_0, text_1)
            energy_cos_sims[i_e, j_e]=cacca.cos_sim
            energy_pvals[i_e, j_e]=cacca.pval
            
    cos_sim_dict[year]['Energy']={}
    cos_sim_dict[year]['Energy']['cos_sim']=energy_cos_sims
    cos_sim_dict[year]['Energy']['pvals']=energy_pvals
    
    utilities_co_s=list(text_dict['Utilities'][s_year].keys())
    utilities_cos_sims=np.zeros((len(utilities_co_s), len(utilities_co_s)))
    utilities_pvals=np.zeros((len(utilities_co_s), len(utilities_co_s)))
    # inside utilities
    for i_e, utilities_co_0 in enumerate(tqdm(utilities_co_s, leave=False, desc='Utilities '+s_year)):
        text_0=text_dict['Utilities'][s_year][utilities_co_0]
        for j_e in range(i_e+1, len(utilities_co_s)):
            utilities_co_1=utilities_co_s[j_e]
            text_1=text_dict['Utilities'][s_year][utilities_co_1]
            cacca=tesste(text_0, text_1)
            utilities_cos_sims[i_e, j_e]=cacca.cos_sim
            utilities_pvals[i_e, j_e]=cacca.pval
            
    cos_sim_dict[year]['Utilities']={}
    cos_sim_dict[year]['Utilities']['cos_sim']=utilities_cos_sims
    cos_sim_dict[year]['Utilities']['pvals']=utilities_pvals


Energy 2015:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings

  0%|          | 0/1000 [00:00<?, ?it/s]


KeyboardInterrupt



### Something I forgot

In [ ]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'rb') as f:
    cos_sim_dict=pickle.load(f)

In [ ]:
for year in range(2015, 2024):
    s_year=str(year)
    for field in ['Energy', 'Utilities']:
        firms=list(text_dict[field][s_year].keys())
        cos_sim_dict[year][field]['companies']=firms

In [ ]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'wb') as f:
    pickle.dump(cos_sim_dict, f)

## Cosine similarity analysis

In [8]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'rb') as f:
    cos_sim_dict=pickle.load(f)

In [9]:
years=list(range(2015, 2024))

In [10]:
# actually, it shouldn't be done, but I saw that relaxing from alpha=0.01 to alpha=0.05,
# I get more validated entries...
alpha=.05

In [11]:
for year in years:
    for field in ['Energy', 'Utilities']:
        pvals=cos_sim_dict[year][field]['pvals']
        cos_sim=cos_sim_dict[year][field]['cos_sim']
        firms=cos_sim_dict[year][field]['companies']
        if len(firms)>1:
            # get the number of tests
            n_tests=len(firms)*(len(firms)-1)/2
            # calculate effective alpha according to Bonferroni
            alpha_bonf=alpha/n_tests
            # get the effective alpha according to fdr
            pvals_vec=np.sort(pvals[pvals>0])
            fdr_vec=(1+np.arange(len(pvals_vec)))*alpha_bonf
            alpha_fdrs=fdr_vec[pvals_vec<=fdr_vec]
            if len(alpha_fdrs)>0:
                alpha_fdr=alpha_fdrs[-1]
                # where are the validated entries?
                where_vals=np.where(np.logical_and(pvals<=alpha_fdr, pvals>0))
                val_couples=[]
                for i in range(len(where_vals[0])):
                    val_couples.append((firms[where_vals[0][i]],firms[where_vals[1][i]]))
                # return values
                print(year, field)
                print('---------------------')
                for i, val_couple in enumerate(val_couples):
                    print(val_couple, cos_sim[where_vals[0][i], where_vals[1][i]])

2019 Utilities
---------------------
('02_SSE_PLC', '05_CENTRICA_PLC') 0.8133289217948914
('02_SSE_PLC', '08_RENEWI_PLC') 0.8001610636711121
('05_CENTRICA_PLC', '08_RENEWI_PLC') 0.7937230467796326
('05_CENTRICA_PLC', '10_GOOD_ENERGY_GROUP_PLC') 0.8450196981430054


In [12]:
cos_sim_dict[2019]['Utilities']['companies']

['02_SSE_PLC', '05_CENTRICA_PLC', '08_RENEWI_PLC', '10_GOOD_ENERGY_GROUP_PLC']

Quite bizarre that only on a single year (btw, before the COVID, i.e. before when I would have expected to see some excess in the cosine similarity) there is such a diffuse similarity. Moreover, CENTRICA is (ehm...) the most central, since it is significantly similar to all others.

In [13]:
validated_cos_sim=(2019, 'Utilities', val_couples)

### SDGs covered and similarities

In [14]:
sdgs_dict={}
for year in trange(2015, 2024):
    s_year=str(year)
    sdgs_dict[year]={}
    for field in ['Energy', 'Utilities']:
        sdgs_dict[year][field]={}
        for key in text_dict[field][s_year].keys():
            text=text_dict[field][s_year][key]
            sdgs_dict[year][field][key]=text2dict(text)

  0%|          | 0/9 [00:00<?, ?it/s]

In [15]:
v_year=validated_cos_sim[0]
v_field=validated_cos_sim[1]

In [16]:
companies=list(sdgs_dict[v_year][v_field].keys())

In [17]:
sdg_superpos=np.zeros((len(companies),len(companies)))
for i_c, company_0 in enumerate(companies):
    sdgs_i_c=list(sdgs_dict[v_year][v_field][company_0].keys())
    for j_c, company_1 in enumerate(companies):
        if i_c!=j_c:
            sdgs_j_c=list(sdgs_dict[v_year][v_field][company_1].keys())
            sdg_superpos[i_c, j_c]=len([sdg for sdg in sdgs_i_c if sdg in sdgs_j_c])/len(sdgs_i_c)

In [18]:
sdg_superpos

array([[0.        , 0.66666667, 0.46666667, 0.73333333],
       [1.        , 0.        , 0.4       , 0.8       ],
       [0.875     , 0.5       , 0.        , 0.75      ],
       [1.        , 0.72727273, 0.54545455, 0.        ]])

Not such a huge similarity. Where does the similarity come from?

In [26]:
def aux(couple):
    dict_0=sdgs_dict[v_year][v_field][couple[0]]
    dict_1=sdgs_dict[v_year][v_field][couple[1]]
    common_sdgs=[key for key in dict_0.keys() if key in dict_1.keys()]
    for sdg in common_sdgs:
        cacca=tesste(dict_0[sdg], dict_1[sdg], tqdm_loops=False)
        
        print('({0:}, {1:}){2:}. Cos={3:.3f}, p-val(theo)={4:.3f}, p-val(sam)={5:.3f},'.format(couple[0][3:], couple[0][3:], sdg.split(':')[0], cacca.cos_sim, cacca.pval, cacca.sampled_pval))

In [24]:
val_couples[0]

('02_SSE_PLC', '05_CENTRICA_PLC')

In [25]:
aux(val_couples[0])

cacca
(SSE_PLC, SSE_PLC)	SDG 3. Cos=0.599, p-val(theo)=0.783, p-val(sam)=0.777,



KeyboardInterrupt



In [29]:
with Pool(20) as pool:
    pool.map(aux, val_couples)

KeyboardInterrupt: 